In [2]:
import numpy as np
from kifmm_py import KiFmm

# Single Precision

In [100]:
np.random.seed(0)

dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
expansion_order = 4 # Single expansion order
n_vec = 1
n_crit = None
depth = 3
if depth:
    expansion_order = [expansion_order]*(depth+1)
n_sources = 1000000
n_targets = 1000000
kernel = "laplace"  # Or 'helmholtz'
field_translation = "blas"  # Or 'fft'
kernel_eval_type = (
    "eval"  # For potentials, or 'eval_deriv' for potentials and gradients
)

# Setup source/target/charge data in Fortran order
sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim)
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim)
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec)
).astype(dtype)

fmm = KiFmm(
    expansion_order,
    sources,
    targets,
    charges,
    kernel_eval_type,
    kernel,
    field_translation,
    prune_empty=True,
    timed=True,
    svd_threshold=None,
    # n_crit=n_crit,
    surface_diff=0,
    depth=depth
)

# Run the FMM
fmm.evaluate()

# Examine result at leaf index
leaf = fmm.target_leaves[1]
found = fmm.potentials(leaf)[0]
targets_leaf = fmm.target_coordinates(leaf)
expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

relative_error = np.abs(expected-found)/expected
# Test
# np.testing.assert_allclose(found, expected, rtol=1e-5)

In [101]:
expansion_order

[4, 4, 4, 4]

In [102]:
max_relative_error = np.max(relative_error)
min_relative_error = np.min(relative_error)
mean_relative_error = np.mean(relative_error)

In [103]:
max_relative_error

0.00021652521

In [104]:
min_relative_error

7.506009e-07

In [105]:
mean_relative_error

4.1538642e-05